In [1]:


import astropy
print('astropy',astropy.__version__ )
from spectral_cube import SpectralCube      # This is a handy package for working with 3D data cubes
from astropy.coordinates import SkyCoord
from reproject import reproject_interp      
from reproject.mosaicking import find_optimal_celestial_wcs 
import regions
import reproject
print('reproject',reproject.__version__)
import spectral_cube
print('spectral_cube',spectral_cube.__version__)
import numpy as np                          
import pylab                                
import matplotlib 
import matplotlib.gridspec as gridspec                                                                                             
import scipy
import astropy.io.fits as fits                                                          
from astropy.wcs import WCS                 
from astropy import units as u              
import pyspeckit as psk   
import astrodendro
import matplotlib.colors as colors
from matplotlib import pyplot as plt
# Suppress warnings we don't care about:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
%matplotlib inline

#%matplotlib widget

astropy 5.1.dev153+gb740594dc
reproject 0.8
spectral_cube 0.6.1.dev22+g003ef16


In [2]:
files = pylab.array(['NGC253.HCO+_4-3.pbcor.fits','NGC253.HCN_4-3.pbcor.fits','NGC253.H13CN_4-3.pbcor.fits','HCN_J1-0.cube.fits','HCOp_J1-0.cube.fits','H13CN_J1-0.cube.fits','ngc253_co32_hires_common.fits', 'CCH_N1-0.cube.fits','HC3N_J11-10_SO_J45-44.cube.fits'])
      
dub='H13COp_J1-0_SiO_J2-1.cube.fits'    


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file).unmasked_copy()    
    #header = fits.getheader(file)
    if i==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    cen_p1 = SkyCoord('00h47m34.034s', '-25d17m29.68s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.234s', '-25d17m09.68s', frame='icrs')
    scW = sc.wcs[:][:][0]
    
    
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = sc[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-500. *u.km / u.s, 5000. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab.to(u.K)
    
    #print(sc_K_kms.wcs)
    #print(sc_K_kms.world[:,0,0][0])
    #print(header)
    if i==0:
        vel, dec, ra = sc_K_kms.world[:,0,0]            # Grab the velocity axis to use for rebinning
        
        NGC_HCO_Plus = sc_K_kms.unmasked_copy()
        NGC_HCO_Plus.write('NGC_HCO_Plus.cube.K-kms.pbcor.fits',overwrite=True)    
        
    if i==1:
        NGC_HCN = sc_K_kms.unmasked_copy()
        NGC_HCN.write('NGC253_HCN.cube.reproject.K-kms.pbcor.fits',overwrite=True)
        del NGC_HCN
    if i==2:
        
        NGC_H13CN = sc_K_kms.unmasked_copy()#This one needs reprojection

del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem
del dec
del ra
del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
NGC_H13CN.allow_huge_operations=True

HCOPH = NGC_HCO_Plus.header


print(NGC_HCO_Plus)
print(NGC_H13CN)

print(type(NGC_HCO_Plus))
print(type(NGC_H13CN))

print(type(NGC_HCO_Plus[0]))
print(type(NGC_H13CN[0]))

print(type(NGC_HCO_Plus[0][0]))
print(type(NGC_H13CN[0][0]))

del NGC_HCO_Plus #rem from mem


NGC_H13CN_reproject = NGC_H13CN.reproject(HCOPH,use_memmap=True)  
del NGC_H13CN #rem from mem
del HCOPH

NGC_HC13N_reproject_interp = NGC_H13CN_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del NGC_H13CN_reproject
NGC_HC13N_reproject_interp.write('NGC_H13CN.cube.reproject.K-kms.pbcor.fits',overwrite=True)      
del NGC_HC13N_reproject_interp         
print("done")        

SpectralCube with shape=(252, 400, 488) and unit=K:
 n_x:    488  type_x: RA---SIN  unit_x: deg    range:    11.884341 deg:   11.891822 deg
 n_y:    400  type_y: DEC--SIN  unit_y: deg    range:   -25.291578 deg:  -25.286036 deg
 n_s:    252  type_s: VRAD-F2F  unit_s: km / s  range:      -27.485 km / s:     600.434 km / s
SpectralCube with shape=(232, 400, 488) and unit=K:
 n_x:    488  type_x: RA---SIN  unit_x: deg    range:    11.884341 deg:   11.891822 deg
 n_y:    400  type_y: DEC--SIN  unit_y: deg    range:   -25.291578 deg:  -25.286036 deg
 n_s:    232  type_s: VRAD-F2F  unit_s: km / s  range:       22.548 km / s:     600.434 km / s
<class 'spectral_cube.spectral_cube.SpectralCube'>
<class 'spectral_cube.spectral_cube.SpectralCube'>
<class 'spectral_cube.lower_dimensional_structures.Slice'>
<class 'spectral_cube.lower_dimensional_structures.Slice'>
<class 'astropy.units.quantity.Quantity'>
<class 'astropy.units.quantity.Quantity'>
done


In [3]:
#The first three cubes haves been reporjected

In [4]:
files = pylab.array(['NGC253.HCO+_4-3.pbcor.fits','NGC253.HCN_4-3.pbcor.fits','NGC253.H13CN_4-3.pbcor.fits','HCN_J1-0.cube.fits','HCOp_J1-0.cube.fits','H13CN_J1-0.cube.fits','ngc253_co32_hires_common.fits', 'CCH_N1-0.cube.fits','HC3N_J11-10_SO_J45-44.cube.fits'])
      
dub='H13COp_J1-0_SiO_J2-1.cube.fits'    


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file)    
    if i==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    cen_p1 = SkyCoord('00h47m34.034s', '-25d17m29.68s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.234s', '-25d17m09.68s', frame='icrs')
    scW = sc.wcs[:][:][0]
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = sc[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-500. *u.km / u.s, 5000. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab.to(u.K)
    
    
    
    if i==0:
        vel, dec, ra = sc_K_kms.world[:,0,0]            # Grab the velocity axis to use for rebinning
        
        NGC_HCO_Plus = sc_K_kms.unmasked_copy()
    if i==3:
        
        NGC_HCN_J1 = sc_K_kms.unmasked_copy()

        
del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem
del dec
del ra
del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
NGC_HCN_J1.allow_huge_operations=True
HCOPH = NGC_HCO_Plus.header
HCOPH['CTYPE3'] = 'VRAD'  #Change from F2F because it seems to be the same thing. There are many incorrect things in these headers, such as incorrect units (comments say m/s where it should be km/s)


del NGC_HCO_Plus #rem from mem

NGC_HCN_J1_reproject = NGC_HCN_J1.reproject(HCOPH,use_memmap=True)  
del NGC_HCN_J1 #rem from mem
del HCOPH
NGC_HCN_J1_reproject_interp = NGC_HCN_J1_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del NGC_HCN_J1_reproject
NGC_HCN_J1_reproject_interp.write('NGC_HCN_J1.cube.reproject.K-kms.pbcor.fits',overwrite=True)      
del NGC_HCN_J1_reproject_interp
print("done")        

done


In [5]:
files = pylab.array(['NGC253.HCO+_4-3.pbcor.fits','NGC253.HCN_4-3.pbcor.fits','NGC253.H13CN_4-3.pbcor.fits','HCN_J1-0.cube.fits','HCOp_J1-0.cube.fits','H13CN_J1-0.cube.fits','ngc253_co32_hires_common.fits', 'CCH_N1-0.cube.fits','HC3N_J11-10_SO_J45-44.cube.fits'])
      
dub='H13COp_J1-0_SiO_J2-1.cube.fits'    


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file)    
    if i==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    cen_p1 = SkyCoord('00h47m34.034s', '-25d17m29.68s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.234s', '-25d17m09.68s', frame='icrs')
    scW = sc.wcs[:][:][0]
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = sc[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-500. *u.km / u.s, 5000. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab.to(u.K)
    
    
    
    if i==0:
        vel, dec, ra = sc_K_kms.world[:,0,0]            # Grab the velocity axis to use for rebinning
        
        NGC_HCO_Plus = sc_K_kms.unmasked_copy()
    if i==4:
        #Make cubes out of the double fits

        sc = SpectralCube.read(dub)
        regpix = regions.RectanglePixelRegion(regions.PixCoord(int(sc.shape[1]/2), int(sc.shape[1]/2)), width=int(sc.shape[1]-1), height=int(sc.shape[1]-1))  
        regpix.meta['range'] = [-100 * u.km/u.s, 318 * u.km/u.s] # spectral range 
        regpix.meta['restfreq'] = 86.84696000*u.GHz # rest frequency of SiJ2-1
        regpix.meta['veltype'] = 'OPTICAL' # velocity convention 
        scb = sc.subcube_from_regions([regpix])  #Make the subcube with the specified frequency
        cubeSi = scb.with_spectral_unit(u.km/u.s,velocity_convention="radio",rest_value=86.84696000*u.GHz)


        scW = cubeSi.wcs[:][:][0]
        y, x = cubeSi.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
        subcubeSi = cubeSi[:,int(x)-50:int(x)+50,int(y)-50:int(y)+50]



        sc_kms = subcubeSi.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
        sc_slab = sc_kms.spectral_slab(-50000. *u.km / u.s, 500000. *u.km / u.s)  # Crop out velocities we don't care about
        sc_slab.allow_huge_operations=True
        sc_K_kms = sc_slab.to(u.K)
        SiJ2_1 = sc_K_kms.unmasked_copy()

        #Make H13CoP cube
        regpix = regions.RectanglePixelRegion(regions.PixCoord(int(sc.shape[1]/2), int(sc.shape[1]/2)), width=int(sc.shape[1]-1), height=int(sc.shape[1]-1))  
        regpix.meta['range'] = [100 * u.km/u.s, 318 * u.km/u.s] # spectral range 
        regpix.meta['restfreq'] = 86.75428840*u.GHz # rest frequency of H13CO+_J1-0, 86.75428840 GHz
        regpix.meta['veltype'] = 'OPTICAL' # velocity convention 
        scb = sc.subcube_from_regions([regpix])  #Make the subcube with the specified frequency
        cubeH13CoP = scb.with_spectral_unit(u.km/u.s,velocity_convention="radio",rest_value=86.75428840*u.GHz)
        scW = cubeH13CoP.wcs[:][:][0]


        y, x = cubeH13CoP.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
        subcubeH13CoP = cubeH13CoP

        sc_kms = subcubeH13CoP.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
        sc_slab = sc_kms.spectral_slab(-50000. *u.km / u.s, 500000. *u.km / u.s)  # Crop out velocities we don't care about
        sc_slab.allow_huge_operations=True
        sc_K_kms = sc_slab.to(u.K)
        H13CoPJ1_0 = sc_K_kms.unmasked_copy()
        NGC_HCOP_J1 = sc_K_kms.unmasked_copy()

        
del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem
del dec
del ra
del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
NGC_HCOP_J1.allow_huge_operations=True
HCOPH = NGC_HCO_Plus.header
HCOPH['CTYPE3'] = 'VRAD'
del NGC_HCO_Plus #rem from mem

NGC_HCOP_J1_reproject = NGC_HCOP_J1.reproject(HCOPH,use_memmap=True)  
del NGC_HCOP_J1 #rem from mem
del HCOPH
NGC_HCOP_J1_reproject_interp = NGC_HCOP_J1_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del  NGC_HCOP_J1_reproject
NGC_HCOP_J1_reproject_interp.write('NGC_HCOP_J1_0.cube.reproject.K-kms.pbcor.fits',overwrite=True)      
del NGC_HCOP_J1_reproject_interp
print("done")              

done


In [6]:
files = pylab.array(['NGC253.HCO+_4-3.pbcor.fits','NGC253.HCN_4-3.pbcor.fits','NGC253.H13CN_4-3.pbcor.fits','HCN_J1-0.cube.fits','HCOp_J1-0.cube.fits','H13CN_J1-0.cube.fits','ngc253_co32_hires_common.fits', 'CCH_N1-0.cube.fits','HC3N_J11-10_SO_J45-44.cube.fits'])
      
dub='H13COp_J1-0_SiO_J2-1.cube.fits'    


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file)    
    if i==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    cen_p1 = SkyCoord('00h47m34.034s', '-25d17m29.68s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.234s', '-25d17m09.68s', frame='icrs')
    scW = sc.wcs[:][:][0]
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = sc[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-500. *u.km / u.s, 5000. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab.to(u.K)
    
    
    
    
    
    if i==0:
        vel, dec, ra = sc_K_kms.world[:,0,0]            # Grab the velocity axis to use for rebinning
        
        NGC_HCO_Plus = sc_K_kms.unmasked_copy()
        
    if i==5:
        NGC_H13CN_J1 = sc_K_kms.unmasked_copy()

        
del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem
del dec
del ra
del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
NGC_H13CN_J1.allow_huge_operations=True
HCOPH = NGC_HCO_Plus.header
HCOPH['CTYPE3'] = 'VRAD'
del NGC_HCO_Plus #rem from mem

NGC_H13CN_J1_reproject = NGC_H13CN_J1.reproject(HCOPH,use_memmap=True)  
del NGC_H13CN_J1 #rem from mem
del HCOPH
NGC_H13CN_J1_reproject_interp = NGC_H13CN_J1_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del NGC_H13CN_J1_reproject
NGC_H13CN_J1_reproject_interp.write('NGC_H13CN_J1_0.cube.reproject.K-kms.pbcor.fits',overwrite=True)  
del NGC_H13CN_J1_reproject_interp        
    
     
print("done")             

done


In [7]:
files = pylab.array(['NGC253.HCO+_4-3.pbcor.fits','NGC253.HCN_4-3.pbcor.fits','NGC253.H13CN_4-3.pbcor.fits','HCN_J1-0.cube.fits','HCOp_J1-0.cube.fits','H13CN_J1-0.cube.fits','ngc253_co32_hires_common.fits', 'CCH_N1-0.cube.fits','HC3N_J11-10_SO_J45-44.cube.fits'])
      
dub='H13COp_J1-0_SiO_J2-1.cube.fits'    


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file)    
    if i==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    cen_p1 = SkyCoord('00h47m34.034s', '-25d17m29.68s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.234s', '-25d17m09.68s', frame='icrs')
    scW = sc.wcs[:][:][0]
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = sc[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-500. *u.km / u.s, 5000. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab.to(u.K)
    
    
    
    if i==0:
        vel, dec, ra = sc_K_kms.world[:,0,0]            # Grab the velocity axis to use for rebinning
        
        NGC_HCO_Plus = sc_K_kms.unmasked_copy()
    if i==6:
        NGC_CO_J3_2 = sc_K_kms.unmasked_copy()

        
del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem
del dec
del ra
del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
NGC_CO_J3_2.allow_huge_operations=True
HCOPH = NGC_HCO_Plus.header
HCOPH['CTYPE3'] = 'VRAD'
del NGC_HCO_Plus #rem from mem

NGC_CO_J3_2_reproject = NGC_CO_J3_2.reproject(HCOPH,use_memmap=True)  
del NGC_CO_J3_2 #rem from mem
del HCOPH

NGC_CO_J3_2_reproject_interp = NGC_CO_J3_2_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del NGC_CO_J3_2_reproject
NGC_CO_J3_2_reproject_interp.write('NGC_CO_J3_2.cube.reproject.K-kms.pbcor.fits',overwrite=True)      
del NGC_CO_J3_2_reproject_interp  

print("done")         

done


In [8]:
files = pylab.array(['NGC253.HCO+_4-3.pbcor.fits','NGC253.HCN_4-3.pbcor.fits','NGC253.H13CN_4-3.pbcor.fits','HCN_J1-0.cube.fits','HCOp_J1-0.cube.fits','H13CN_J1-0.cube.fits','ngc253_co32_hires_common.fits', 'CCH_N1-0.cube.fits','HC3N_J11-10_SO_J45-44.cube.fits'])
      
dub='H13COp_J1-0_SiO_J2-1.cube.fits'    


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file)    
    if i==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    cen_p1 = SkyCoord('00h47m34.034s', '-25d17m29.68s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.234s', '-25d17m09.68s', frame='icrs')
    scW = sc.wcs[:][:][0]
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = sc[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-500. *u.km / u.s, 5000. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab.to(u.K)
    
    
    
    if i==0:
        vel, dec, ra = sc_K_kms.world[:,0,0]            # Grab the velocity axis to use for rebinning
        
        NGC_HCO_Plus = sc_K_kms.unmasked_copy()
    if i==7:
        CCH_N1 = sc_K_kms.unmasked_copy()

        
del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem
del dec
del ra
del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
CCH_N1.allow_huge_operations=True
HCOPH = NGC_HCO_Plus.header
HCOPH['CTYPE3'] = 'VRAD'
del NGC_HCO_Plus #rem from mem

CCH_N1_reproject =CCH_N1.reproject(HCOPH,use_memmap=True)  
del CCH_N1 #rem from mem
del HCOPH
CCH_N1_reproject_interp = CCH_N1_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del CCH_N1_reproject
CCH_N1_reproject_interp.write('CCH_N1.cube.reproject.K-kms.pbcor.fits',overwrite=True)   
del CCH_N1_reproject_interp         
print("done")            

done


In [9]:
files = pylab.array(['NGC253.HCO+_4-3.pbcor.fits','NGC253.HCN_4-3.pbcor.fits','NGC253.H13CN_4-3.pbcor.fits','HCN_J1-0.cube.fits','HCOp_J1-0.cube.fits','H13CN_J1-0.cube.fits','ngc253_co32_hires_common.fits', 'CCH_N1-0.cube.fits',"HC3N_11_10.fits"])
      
dub='H13COp_J1-0_SiO_J2-1.cube.fits'    


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file)    
    if i==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    cen_p1 = SkyCoord('00h47m34.034s', '-25d17m29.68s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.234s', '-25d17m09.68s', frame='icrs')
    scW = sc.wcs[:][:][0]
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = sc[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-500. *u.km / u.s, 5000. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab.to(u.K)
    
    
    
    if i==0:
        vel, dec, ra = sc_K_kms.world[:,0,0]            # Grab the velocity axis to use for rebinning
        
        NGC_HCO_Plus = sc_K_kms.unmasked_copy()
    if i==8:
        HC3N_J11 = sc_K_kms.unmasked_copy()

        
del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem
del dec
del ra
del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
HC3N_J11.allow_huge_operations=True
HCOPH = NGC_HCO_Plus.header
HCOPH['CTYPE3'] = 'VRAD'
del NGC_HCO_Plus #rem from mem

HC3N_J11_reproject =HC3N_J11.reproject(HCOPH,use_memmap=True)  
del HC3N_J11 #rem from mem
del HCOPH
HC3N_J11_reproject_interp = HC3N_J11_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del HC3N_J11_reproject
HC3N_J11_reproject_interp.write('HC3N_J11.cube.reproject.K-kms.pbcor.fits',overwrite=True) 
del HC3N_J11_reproject_interp          
print("done")        

done


In [3]:

      
files = pylab.array(['NGC253.HCO+_4-3.pbcor.fits', "H13CO_Plus_1_0.fits"])


for i,file in enumerate(files):
                    
    sc = SpectralCube.read(file)  
    
    cen_p1 = SkyCoord('00h47m34.034s', '-25d17m29.68s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.234s', '-25d17m09.68s', frame='icrs')
    scW = sc.wcs[:][:][0]
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = sc[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-500. *u.km / u.s, 5000. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab.to(u.K)
    
    
    
    if i==0:
        vel, dec, ra = sc_K_kms.world[:,0,0]            # Grab the velocity axis to use for rebinning
        
        NGC_HCO_Plus = sc_K_kms.unmasked_copy()
    if i==1:
        H13CoPJ1_0 = sc_K_kms.unmasked_copy()

        
del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem
del dec
del ra
del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
H13CoPJ1_0.allow_huge_operations=True
HCOPH = NGC_HCO_Plus.header
HCOPH['CTYPE3'] = 'VRAD'
del NGC_HCO_Plus #rem from mem

H13CoPJ1_0_reproject = H13CoPJ1_0.reproject(HCOPH,use_memmap=True)  
del H13CoPJ1_0 #rem from mem
del HCOPH
H13CoPJ1_0_reproject_interp = H13CoPJ1_0_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del H13CoPJ1_0_reproject
H13CoPJ1_0_reproject_interp.write('H13CoPJ1_0.cube.reproject.K-kms.pbcor.fits',overwrite=True)      
del H13CoPJ1_0_reproject_interp        
           
print("done")        

done


In [11]:
files = pylab.array(['NGC253.HCO+_4-3.pbcor.fits', "SiO_2_1.fits"])


for i,file in enumerate(files):
                    
    sc = SpectralCube.read(file)  
    
    cen_p1 = SkyCoord('00h47m34.034s', '-25d17m29.68s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.234s', '-25d17m09.68s', frame='icrs')
    scW = sc.wcs[:][:][0]
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = sc[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-500. *u.km / u.s, 5000. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab.to(u.K)
    
    
    
    if i==0:
        vel, dec, ra = sc_K_kms.world[:,0,0]            # Grab the velocity axis to use for rebinning
        
        NGC_HCO_Plus = sc_K_kms.unmasked_copy()
    if i==1:
        SiJ2_1 = sc_K_kms.unmasked_copy()

        
del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem
del dec
del ra
del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
SiJ2_1.allow_huge_operations=True
HCOPH = NGC_HCO_Plus.header
HCOPH['CTYPE3'] = 'VRAD'
del NGC_HCO_Plus #rem from mem

SiJ2_1_0_reproject = SiJ2_1.reproject(HCOPH,use_memmap=True)  
del SiJ2_1 #rem from mem
del HCOPH
SiJ2_1_reproject_interp = SiJ2_1_0_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del SiJ2_1_0_reproject
SiJ2_1_reproject_interp.write('SiJ2_1.cube.reproject.K-kms.pbcor.fits',overwrite=True)       

del SiJ2_1_reproject_interp        
          
    

    
    
    
print("done")

done


In [ ]:
file1 = 'NGC253.HCO+_4-3.pbcor.fits'

file2  ='H8_continuum.pbcor.fits'


for i,file in enumerate(files):
                    
    sc = SpectralCube.read(file)  
    
    cen_p1 = SkyCoord('00h47m34.034s', '-25d17m29.68s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.234s', '-25d17m09.68s', frame='icrs')
    scW = sc.wcs[:][:][0]
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = sc[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-500. *u.km / u.s, 5000. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab.to(u.K)
    
    
    
    if i==0:
        vel, dec, ra = sc_K_kms.world[:,0,0]            # Grab the velocity axis to use for rebinning
        
        NGC_HCO_Plus = sc_K_kms.unmasked_copy()
    if i==1:
        SiJ2_1 = sc_K_kms.unmasked_copy()

        
del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem
del dec
del ra
del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
SiJ2_1.allow_huge_operations=True
HCOPH = NGC_HCO_Plus.header
HCOPH['CTYPE3'] = 'VRAD'
del NGC_HCO_Plus #rem from mem

SiJ2_1_0_reproject = SiJ2_1.reproject(HCOPH,use_memmap=True)  
del SiJ2_1 #rem from mem
del HCOPH
SiJ2_1_reproject_interp = SiJ2_1_0_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del SiJ2_1_0_reproject
SiJ2_1_reproject_interp.write('SiJ2_1.cube.reproject.K-kms.pbcor.fits',overwrite=True)       

del SiJ2_1_reproject_interp        
          
    

    
    
    
print("done")

In [36]:
files = pylab.array(['NGC253.HCO+_4-3.pbcor.fits','ngc253_850_micron_dust.fits'])



for i,file in enumerate(files):
    
    if i==0:
        sc = SpectralCube.read(file) 
        cen_p1 = SkyCoord('00h47m34.034s', '-25d17m29.68s', frame='icrs')
        cen_p2 = SkyCoord('00h47m32.234s', '-25d17m09.68s', frame='icrs')
        scW = sc.wcs[:][:][0]
        raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
        raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
        subcube = sc[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    if i==1:
        subcube = spectral_cube.Projection.from_hdu(fits.open(file)[0])
    #print(subcube.hdu.header)
    
    
    #Put in the right system
    
    subcube.allow_huge_operations=True
    
    
    
    if i==0:
        vel, dec, ra = subcube.world[:,0,0]            # Grab the velocity axis to use for rebinning
        
        NGC_HCO_Plus = subcube.moment0()
    if i==1:
        Cont = subcube
        

        
del dec
del ra
Cont.allow_huge_operations=True
#HCOPH = NGC_HCO_Plus.header
HCOPH = NGC_HCO_Plus.hdu.header
#HCOPH['NAXIS'] = 2
print(HCOPH)
#print([HCOPH['NAXIS{0}'.format(i + 1)] for i in range(HCOPH['NAXIS'])][::-1])
print([Cont.hdu.header['NAXIS{0}'.format(i + 1)] for i in range(Cont.hdu.header['NAXIS'])][::-1])
print(np.shape(NGC_HCO_Plus))
del NGC_HCO_Plus #rem from mem

Cont_reproject =Cont.reproject(HCOPH)  
del HCOPH

Cont_reproject.write('Continuum_Reproject.fits',overwrite=True) 
print(np.shape(Cont_reproject))
del Cont_reproject          
print("done")        

SIMPLE  =                    T / conforms to FITS standard                      BITPIX  =                  -32 / array data type                                NAXIS   =                    2 / number of array dimensions                     NAXIS1  =                  488                                                  NAXIS2  =                  400                                                  BMAJ    =   4.722222222222E-05                                                  BMIN    =   3.611111111111E-05                                                  BPA     =   -73.31999999999999                                                  BTYPE   = 'Intensity'                                                           OBJECT  = 'NGC253  '                                                            BUNIT   = 'K km s-1'           / Brightness (pixel) unit                        ALTRVAL =   1.642648371622E+06                                                  ALTRPIX =   1.000000000000E+00          